## 12.3 自动并行
- **目录**
  - 12.3.1 基于GPU的并行计算
  - 12.3.2 并行计算与通信

- 深度学习框架（例如，MxNet、飞桨和PyTorch）会在后端**自动构建计算图**。
- 利用计算图，系统可以了解所有**依赖关系**，并且可以**选择性地并行执行多个不相互依赖的任务以提高速度**。
  - 例如， 12.2节中的 图12.2.2独立初始化两个变量。
  - 因此，系统可以选择并行执行它们。
- 通常情况下单个操作符将使用所有CPU或单个GPU上的所有计算资源。
  - 例如，即使在一台机器上有多个CPU处理器，dot操作符也将使用所有CPU上的所有核心（和线程）。
  - 这样的行为同样适用于单个GPU。
- 因此，并行化对单设备计算机来说并不是很有用，而并行化对于多个设备就很重要了。
- 虽然并行化通常应用在多个GPU之间，但增加本地CPU以后还将提高少许性能。
- 借助**自动并行化框架**的便利性，我们可以依靠几行Python代码实现相同的目标。
- 对自动并行计算的讨论主要集中在使用CPU和GPU的并行计算上，以及计算和通信的并行化内容。
- 请注意，本节中的实验至少需要两个GPU来运行。

In [1]:
import torch
from d2l import torch as d2l

## 12.3.1 基于GPU的并行计算

- 让我们从定义一个具有参考性的用于测试的工作负载开始：
  - 下面的`run`函数将执行$10$ 次“矩阵－矩阵”乘法时需要使用的数据分配到两个变量（`x_gpu1`和`x_gpu2`）中，
  - 这两个变量分别位于我们选择的不同设备上。


In [2]:
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
x_gpu2 = torch.rand(size=(4000, 4000), device=devices[0])

- 现在使用函数来计算数据。
- 通过在测量之前预热设备（对设备执行一次传递）来确保缓存的作用不影响最终的结果。 
- `torch.cuda.synchronize()`函数将会等待一个CUDA设备上的所有流中的所有核心的计算完成。
- 函数接受一个`device`参数，代表是哪个设备需要同步。
- 如果device参数是`None`（默认值），它将使用`current_device()`找出的当前设备。


In [5]:
run(x_gpu1)
run(x_gpu2)  # 预热设备
torch.cuda.synchronize(devices[0])
torch.cuda.synchronize(devices[0])

with d2l.Benchmark('GPU1 time'):
    run(x_gpu1)
    torch.cuda.synchronize(devices[0])

with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    torch.cuda.synchronize(devices[0])

GPU1 time: 7.2362 sec
GPU2 time: 7.2460 sec


- 如果删除两个任务之间的`synchronize`语句，系统就可以在两个设备上自动实现并行计算。


In [3]:
with d2l.Benchmark('GPU1 & GPU2'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

GPU1 & GPU2: 15.6434 sec


- 在上述情况下，总执行时间小于两个部分执行时间的总和，因为深度学习框架自动调度两个GPU设备上的计算，而不需要用户编写复杂的代码。

## 12.3.2 并行计算与通信
- 在许多情况下，需要在不同的设备之间移动数据，比如在CPU和GPU之间，或者在不同的GPU之间。
- 例如，当打算执行分布式优化时，就需要移动数据来聚合多个加速卡上的梯度。
- 通过在GPU上计算，然后将结果复制回CPU来模拟这个过程。

In [5]:
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

with d2l.Benchmark('在GPU1上运行'):
    y = run(x_gpu1)
    torch.cuda.synchronize()

with d2l.Benchmark('复制到CPU'):
    y_cpu = copy_to_cpu(y)
    torch.cuda.synchronize()

在GPU1上运行: 0.5022 sec
复制到CPU: 3.0686 sec


- 上述这种执行方式效率不高。
- 注意到当列表中的其余部分还在计算时，可能就已经开始将`y`的部分复制到CPU了。
  - 例如，当计算一个小批量的（反传）梯度时。某些参数的梯度将比其他参数的梯度更早可用。
- 因此，在GPU仍在运行时就开始使用PCI-Express总线带宽来移动数据是有利的。
- 在PyTorch中，`to()`和`copy_()`等函数都允许显式的`non_blocking`参数，这允许在不需要同步时调用方可以绕过同步。
- 设置`non_blocking=True`来模拟这个场景。


In [6]:
with d2l.Benchmark('在GPU1上运行并复制到CPU'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y, True)
    torch.cuda.synchronize()

在GPU1上运行并复制到CPU: 2.1492 sec


- 两个操作所需的总时间少于它们各部分操作所需时间的总和。
- 请注意，与并行计算的区别是通信操作使用的资源：CPU和GPU之间的总线。
- 事实上，我们可以在两个设备上同时进行计算和通信。
- 如上所述，计算和通信之间存在的依赖关系是必须先计算y[i]，然后才能将其复制到CPU。
- 幸运的是，系统可以在计算y[i]的同时复制y[i-1]，以减少总的运行时间。

- 最后，本节给出了一个简单的两层多层感知机在CPU和两个GPU上训练时的计算图及其依赖关系的例子，如 图12.3.1所示。
- 手动调度由此产生的并行程序将是相当痛苦的。这就是基于图的计算后端进行优化的优势所在。


<center><img src='../img/twogpu.svg'></center>
<center>图12.3.1 在一个CPU和两个GPU上的两层的多层感知机的计算图及其依赖关系</center>

## 小结

* 现代系统拥有多种设备，如多个GPU和多个CPU，还可以并行地、异步地使用它们。
* 现代系统还拥有各种通信资源，如PCI Express、存储（通常是固态硬盘或网络存储）和网络带宽，为了达到最高效率可以并行使用它们。
* 后端可以通过自动化地并行计算和通信来提高性能。